### Clean and Transform Spotify Data

- Takes data from each sheet in the SpotifyPlaylistData excel file and creates a dataframe.
- Creates the genre column to merge the playlists.

In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time

SpotifyExcelFile = '../Spotify/Line4LinePlaylistData_20230307.xlsx'

def cleanPlaylistData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)

  df = df.drop(columns=['Unnamed: 0', 'duration_ms'])

  df = df.loc[:,['track_uri','artist_uri', 'album_uri', 'track_name', 'artist_name', 'album_name', 'release_date', 'explicit', 'popularity']]
  # df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')
  df = df.reset_index(drop=True)
  
  return df

def cleanArtistData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.reset_index(drop=True)
  
  return df

def cleanAlbumData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0', 'artist_uri'])
  df = df.loc[:,['album_uri', 'album_name', 'artist_name', 'release_date', 'label', 'total_tracks', 'popularity']]
  df = df.reset_index(drop=True)
  
  return df

def cleanTrackFeaturesData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df.rename(columns={'uri': 'track_uri'}, inplace=True)
  df = df.reset_index(drop=True)
  
  return df


def mergeDataFrames(HHDF, CHHDF):
  mergedDF = pd.concat([HHDF, CHHDF], ignore_index=True)

  return mergedDF

HHPlaylistSpotDF = cleanPlaylistData('HHPlaylist')
CHHPlaylistSpotDF = cleanPlaylistData('CHHPlaylist')
HHPlaylistSpotDF.insert(7, 'genre', 'Hip-Hop')
CHHPlaylistSpotDF.insert(7, 'genre', 'Christian Hip-Hop')

PlaylistSpotDF = mergeDataFrames(HHPlaylistSpotDF, CHHPlaylistSpotDF)

HHTopTracksSpotDF = cleanPlaylistData('HHTopTracks')
CHHTopTracksSpotDF = cleanPlaylistData('CHHTopTracks')
HHTopTracksSpotDF.insert(7, 'genre', 'Hip-Hop')
CHHTopTracksSpotDF.insert(7, 'genre', 'Christian Hip-Hop')

TopTracksSpotDF = mergeDataFrames(HHTopTracksSpotDF, CHHTopTracksSpotDF)

HHArtistsSpotDF = cleanArtistData('HHArtists')
CHHArtistsSpotDF = cleanArtistData('CHHArtists')

ArtistsSpotDF = mergeDataFrames(HHArtistsSpotDF, CHHArtistsSpotDF)

HHAlbumsSpotDF = cleanAlbumData('HHAlbums')
CHHAlbumsSpotDF = cleanAlbumData('CHHAlbums')

AlbumsSpotDF = mergeDataFrames(HHAlbumsSpotDF, CHHAlbumsSpotDF)

HHTrackFeaturesSpotDF = cleanTrackFeaturesData('HHTrackFeatures')
CHHTrackFeaturesSpotDF = cleanTrackFeaturesData('CHHTrackFeatures')

TrackFeaturesSpotDF = mergeDataFrames(HHTrackFeaturesSpotDF, CHHTrackFeaturesSpotDF)

### Create Excel File

- Compiles the json files created from each API call into TransformedMusicData excel file.
- The file is used in the Tableau project TrackStarz Music Analysis:
(Pending)

In [ ]:
# Push data to excel

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import datetime as datetime

xlxWriter = pd.ExcelWriter('TransformedLine4LineData.xlsx', engine='xlsxwriter')

PlaylistSpotDF.to_excel(xlxWriter, sheet_name='Playlist')
TopTracksSpotDF.to_excel(xlxWriter, sheet_name='TopTracks')
ArtistsSpotDF.to_excel(xlxWriter, sheet_name='Artists')
AlbumsSpotDF.to_excel(xlxWriter, sheet_name='Albums')
TrackFeaturesSpotDF.to_excel(xlxWriter, sheet_name='TrackFeatures')

xlxWriter.close()